# Snowflake SQL Alchemy

### Uploading the web-scaped and cleaned data using pydantic 'items.csv' into snowflake using Snowflake-SQLAlchemy into our created db

In [ ]:
pip install python-dotenv

In [ ]:
pip install bcrypt

In [ ]:
pip install snowflake-sqlalchemy

In [7]:
#!/usr/bin/env python
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
from sqlalchemy import create_engine
import bcrypt
import os

### Snowflake Testing environment setup & data upload

In [8]:
create_test_stage = """CREATE STAGE TEST_URL_STAGING DIRECTORY = ( ENABLE = true );"""

In [9]:
create_test_table = """CREATE OR REPLACE TABLE test_cfa_courses (
        NameOfTopic STRING,
        Title STRING,
        Year INTEGER,
        Level STRING,
        Introduction STRING,
        LearningOutcome STRING,
        LinkToPDF STRING,
        LinkToSummary STRING
        );"""

In [10]:
upload_to_test_stage = """PUT file://..\data\Validation_data.csv @PC_DBT_DB.public.TEST_URL_STAGING;"""

In [11]:
copy_stage_to_test_table = """COPY INTO test_cfa_courses
  FROM @PC_DBT_DB.public.TEST_URL_STAGING
  FILE_FORMAT = (type = csv field_optionally_enclosed_by='"' skip_header = 1)
  PATTERN = 'Validation_data.csv.gz'
  ON_ERROR = 'skip_file';"""

### Snowflake Production Environment setup & data upload

In [12]:
create_prod_stage = """CREATE STAGE URL_STAGING DIRECTORY = ( ENABLE = true );"""

In [13]:
create_prod_table_query = """CREATE OR REPLACE TABLE cfa_courses (
        NameOfTopic STRING,
        Title STRING,
        Year INTEGER,
        Level STRING,
        Introduction STRING,
        LearningOutcome STRING,
        LinkToPDF STRING,
        LinkToSummary STRING
        );"""

In [14]:
upload_to_prod_stage = """PUT file://..\data\Validation_data.csv @PC_DBT_DB.public.URL_STAGING;"""

In [15]:
copy_stage_to_prod_table = """COPY INTO cfa_courses
  FROM @PC_DBT_DB.public.URL_STAGING
  FILE_FORMAT = (type = csv field_optionally_enclosed_by='"' skip_header = 1)
  PATTERN = 'Validation_data.csv.gz'
  ON_ERROR = 'skip_file';"""

In [16]:
load_dotenv()

u=os.getenv("SNOWFLAKE_USER")
p=os.getenv("SNOWFLAKE_PASS")
ai=os.getenv("SNOWFLAKE_ACCOUNTID")


engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/'.format(
        user=u,
        password=p,
        account_identifier=ai,
    )
)



try:
    connection = engine.connect()
    connection.execute("USE DATABASE PC_DBT_DB")
    connection.execute("USE WAREHOUSE PC_DBT_WH")
    
    results = connection.execute(create_test_stage)
    results = connection.execute(create_test_table)
    results = connection.execute(upload_to_test_stage)
    results = connection.execute(copy_stage_to_test_table)
    
    results = connection.execute(create_prod_stage)
    results = connection.execute(create_prod_table_query)
    results = connection.execute(upload_to_prod_stage)
    results = connection.execute(copy_stage_to_prod_table)

finally:
    print("Done")
    connection.close()
    engine.dispose()

Done
